# Princess Diana's Gains Guide

- Quarterly Gains Report #1: Princess Diana's Gains Guide
- Consider tracking the intensity of each set; I.E. % of 1RM, 60%, 75%, 85% etc.
    - This would require calculating 1RMs at the beginning of the workout routine.

In [1]:
#basic imports
import numpy as np
import pandas as pd

#visualization libraries
import cufflinks as cf
cf.go_offline()

import plotly.graph_objs as go
import plotly.tools as tls
import plotly.io as pio
from plotly.offline import iplot

import plotly.io as pio

In [2]:
def gains(df, color, category = 'abs'):
    '''
    Plot workout volume within a routine over time.
    '''
    # make the traces the column names in reverse, placing the total volume column in front for the largest subplot
    traces = df.columns[::-1]
    
    # create subplots, one plot for each exercise in the routine
    if category == 'abs':
        subplots = tls.make_subplots(rows=3, cols=4, 
                              specs=[[{'colspan': 2, 'rowspan': 3}, None, {}, {}], # reserve one large subplot for the total volume field
                                     [None, None, {}, {}], # reserve remaining subplots for other exercises
                                     [None, None, {}, {}],
                                    ],
                                 subplot_titles = traces,
                              print_grid=True)
        # set row and col position list for plotting
        rows = [1,1,1,2,2,3,3]
        cols = [1,3,4,3,4,3,4,]
        x_axes = range(2,6)
        
    else:
        subplots = tls.make_subplots(rows=5, cols=4, 
                              specs=[[{'colspan': 2, 'rowspan': 5}, None, {}, {}], # reserve one large subplot for the total volume field
                                     [None, None, {}, {}], # reserve remaining subplots for other exercises
                                     [None, None, {}, {}],
                                     [None, None, {}, {}],
                                     [None, None, {}, {}]
                                    ],
                                 subplot_titles = traces,
                              print_grid=True)
        
        # set row and col position list for plotting
        rows = [1,1,1,2,2,3,3,4,4,5,5]
        cols = [1,3,4,3,4,3,4,3,4,3,4]
        x_axes = range(2,10)
    
    # intanstiate plot using the subplots
    f = go.FigureWidget(subplots)
    
    # create rolling 5-workout mean for each exercise
    means = df.rolling(5).mean()
    
    # plot each exercise in its own subplot
    for row, col, trace in zip(rows, cols, traces):
        
        # print values for diagnostic purposes
        print(trace)
        print(row, col)
        
        # plot exercise volumes then rolling means
        for frame in [df, means]:
            
            # exercise volumes
            f.add_scatter(x = frame.index, y = frame[trace], 
                          mode = 'lines',
                          marker = {'color':'black'} if frame is means else {'color': f'{color}'},
                          line = {'shape': 'spline'} if frame is means else {}, # make the rolling means smooth but the normal value normal
                          showlegend = False, 
                          name = trace if frame is df else 'Mean',
                          row = row, col= col)
    
    # hide x-axis values for the upper-most plots, leaving on the x-axis of the bottom plots for better visibility
    for axis in x_axes:
        f.layout[f'xaxis{axis}'].visible = False
    
    # add my signature logo
    f.layout.images = [dict(
        source="https://raw.githubusercontent.com/WilliamGreenlaw/Logos/master/signature_min.png",
        xref="paper", yref="paper",
        x=1, y=1.10,
        sizex=0.2, sizey=0.2,
        xanchor="right", yanchor="bottom"
      )]
    
    # set axis labels
    f.layout.yaxis.title = 'Volume (lbs)'
    f.layout.xaxis.title = 'Date'
    
    # set colors
    f.layout.plot_bgcolor = '#E5E5E5'
    f.layout.paper_bgcolor = '#E5E5E5'
    
    return f

In [16]:
exercises = pd.read_csv('../../fitnotes/exercises.csv')

In [18]:
categories = ['Chest', 'Back', 'Abs']
chest, back, ab = [exercises
                   .loc[(exercises.routine == 'Princess Diana Gains Guide') & 
                                 (exercises.category == category)]
                   .pivot(index = 'date', columns = 'exercise', values = 'volume')
                   .assign(total_volume = lambda df: df.sum(axis = 1))
                   .rename(columns = {'total_volume': 'Total Volume'})
                   for category in categories]

In [19]:
figs = {}

In [20]:
figs['abs'] = gains(ab, color = '#037F8C', category = 'abs')
figs['abs'].layout.title = '<b>Ab Volume Rises Because of Weighted Exercises Added Later on</b><br><i>Ab Volume over Time, Five-Workout Average in Black'

This is the format of your plot grid:
[ (1,1) x1,y1           -      ]  [ (1,3) x2,y2 ]  [ (1,4) x3,y3 ]
       |             (empty)      [ (2,3) x4,y4 ]  [ (2,4) x5,y5 ]
       |                |         [ (3,3) x6,y6 ]  [ (3,4) x7,y7 ]

Total Volume
1 1
Mounted Oblique Leans
1 3
Med Ball Straight Leg Spread Eagles
1 4
Hanging Knee Raise
2 3
Hanging Front Snap Kick Raise
2 4
Decline Oblique Chops
3 3
Ab-Wheel Rollout Side-Side-Front
3 4


In [21]:
figs['chest'] = gains(chest, color = '#A6431F', category = 'chest')
figs['chest'].layout.title = '<b>Chest Volume Has Declined Since February 1st, 2018</b><br><i>Chest Volume over Time, Five-Workout Average in Black'

This is the format of your plot grid:
[ (1,1) x1,y1             -        ]  [ (1,3) x2,y2 ]    [ (1,4) x3,y3 ]  
       |               (empty)        [ (2,3) x4,y4 ]    [ (2,4) x5,y5 ]  
       |               (empty)        [ (3,3) x6,y6 ]    [ (3,4) x7,y7 ]  
       |               (empty)        [ (4,3) x8,y8 ]    [ (4,4) x9,y9 ]  
       |                  |           [ (5,3) x10,y10 ]  [ (5,4) x11,y11 ]

Total Volume
1 1
Single Arm Hammer Strength Press
1 3
Lower Chest Bracket Pushups
1 4
Incline Pushup
2 3
Incline Barbell Bench Press
2 4
Flat Barbell Bench Press
3 3
Decline Dumbbell Fly
3 4
Decline Dumbbell Benchpress
4 3
Decline Cable Fly
4 4
Decline Barbell Bench Press
5 3
Close Grip Dip
5 4


In [22]:
figs['back'] = gains(back, color = '#5B4973', category = 'back')
figs['back'].layout.title = '<b>Back Volume Has Remained Stagnant on Average over Time</b><br><i>Back Volume over Time, Five-Workout Average in Black'

This is the format of your plot grid:
[ (1,1) x1,y1             -        ]  [ (1,3) x2,y2 ]    [ (1,4) x3,y3 ]  
       |               (empty)        [ (2,3) x4,y4 ]    [ (2,4) x5,y5 ]  
       |               (empty)        [ (3,3) x6,y6 ]    [ (3,4) x7,y7 ]  
       |               (empty)        [ (4,3) x8,y8 ]    [ (4,4) x9,y9 ]  
       |                  |           [ (5,3) x10,y10 ]  [ (5,4) x11,y11 ]

Total Volume
1 1
Wide Lat Pulldown Supinated
1 3
Single Arm Lat Cable Pulldown
1 4
Seated Back Extension
2 3
Reverse Cable Fly
2 4
Pull Up
3 3
Decline Pullups
3 4
Bent Over Single Arm Cable Rows with Twist
4 3
Bent Over Rope Pulldowns
4 4
Bent Over EZ Bar Cable Rows
5 3
Barbell Row
5 4


In [23]:
figs['back']

FigureWidget({
    'data': [{'marker': {'color': '#5B4973'},
              'mode': 'lines',
              'nam…

In [27]:
figs['chest']

FigureWidget({
    'data': [{'marker': {'color': '#A6431F'},
              'mode': 'lines',
              'nam…

In [25]:
figs['abs']

FigureWidget({
    'data': [{'marker': {'color': '#037F8C'},
              'mode': 'lines',
              'nam…

In [26]:
pio.write_image(figs['back'], 'back.svg', width = 1500, height = 500)
pio.write_image(figs['chest'], 'chest.svg', width = 1500, height = 500)
pio.write_image(figs['abs'], 'abs.svg', width = 1500, height = 500)